In [1]:
import numpy as np
import antea.database.load_db as db
from antea.io.mc_io import load_mcsns_response
from antea.io.mc_io import load_mcTOFsns_response
from antea.io.mc_io import load_mcparticles
from antea.io.mc_io import load_mchits

import antea.reco.reco_functions   as rf

In [2]:
DataSiPM     = db.DataSiPM('petalo', 0)
DataSiPM_idx = DataSiPM.set_index('SensorID')

filename     = '/Users/carmenromoluque/nexus_petit_analysis/PETit-ring/ring_dataframes_analysis/petit_ring_tof_all_tables.000.pet.h5'
sns_response = load_mcsns_response(filename)
print(type(sns_response.sensor_id[0]))
print(type(np.array([1000, 1001])[0]))

<class 'numpy.uint32'>
<class 'numpy.int64'>


In [3]:
evt_sns = sns_response[sns_response.event_id==31]
sipms   = DataSiPM_idx.loc[evt_sns.sensor_id]
sns_ids1 = sipms.index.values
sns_ids2 = sipms.index.astype('int64').values
print(-sns_ids1)
print(type(sns_ids1[0]))
print(type(sns_ids2[0]))
sns_ids1 + (-sns_ids1)

for i in (-sns_ids1):
    if i > 0:
        print('Tengo razón')


[18446744073709547527 18446744073709547877 18446744073709548577
 18446744073709548049 18446744073709547880 18446744073709548576
 18446744073709548225 18446744073709548054 18446744073709547198
 18446744073709547874 18446744073709547529 18446744073709548226
 18446744073709548926 18446744073709548233 18446744073709548232
 18446744073709548047 18446744073709547701 18446744073709548402
 18446744073709548394 18446744073709548048 18446744073709548400
 18446744073709547875 18446744073709548757 18446744073709548229
 18446744073709548556 18446744073709547881 18446744073709548053
 18446744073709548052 18446744073709547522 18446744073709547528
 18446744073709547699 18446744073709547524 18446744073709547703
 18446744073709548051 18446744073709548224 18446744073709547371
 18446744073709548056 18446744073709548207 18446744073709548572
 18446744073709548223 18446744073709548230 18446744073709550482
 18446744073709548401 18446744073709548754 18446744073709548222
 18446744073709547878 184467440737095487

In [10]:
def reconstruct_coincidences(sns_response, tof_response, charge_range, DataSiPM_idx, particles, hits):
    if 'SensorID' in DataSiPM_idx.columns:
        DataSiPM_idx = DataSiPM_idx.set_index('SensorID')

    max_sns = sns_response[sns_response.charge == sns_response.charge.max()]
    ## If by chance two sensors have the maximum charge, choose one (arbitrarily)
    if len(max_sns != 1):
        max_sns = max_sns[max_sns.sensor_id == max_sns.sensor_id.min()]
    max_sipm = DataSiPM_idx.loc[max_sns.sensor_id]
    max_pos  = np.array([max_sipm.X.values, max_sipm.Y.values, max_sipm.Z.values]).transpose()[0]

    sipms         = DataSiPM_idx.loc[sns_response.sensor_id]
    #sns_ids       = sipms.index.astype('int64').values
    sns_ids       = sipms.index.values
    sns_positions = np.array([sipms.X.values, sipms.Y.values, sipms.Z.values]).transpose()
    sns_charges   = sns_response.charge

    sns1, sns2, pos1, pos2, q1, q2 = rf.divide_sipms_in_two_hemispheres(sns_ids, sns_positions, sns_charges, max_pos)

    sns1 = - np.array(sns1)
    sns2 = - np.array(sns2)

    tot_q1 = sum(q1)
    tot_q2 = sum(q2)

    sel1 = (tot_q1 > charge_range[0]) & (tot_q1 < charge_range[1])
    sel2 = (tot_q2 > charge_range[0]) & (tot_q2 < charge_range[1])
    if not sel1 or not sel2:
        return [], [], [], [], None, None, None, None, None, None, None, None

    ### TOF
    min1, min_tof1 = find_first_time_of_sensors(tof_response, sns1)
    min2, min_tof2 = find_first_time_of_sensors(tof_response, sns2)

    true_pos1, true_pos2, true_t1, true_t2, _, _ = rf.find_first_interactions_in_active(particles, hits)

    if not len(true_pos1) or not len(true_pos2):
        print("Cannot find two true gamma interactions for this event")
        return [], [], [], [], None, None, None, None, None, None, None, None

    scalar_prod = true_pos1.dot(max_pos)
    if scalar_prod > 0:
        int_pos1 = pos1
        int_pos2 = pos2
        int_q1   = q1
        int_q2   = q2
        int_min1 = min1
        int_min2 = min2
        int_tof1 = min_tof1
        int_tof2 = min_tof2
    else:
        int_pos1 = pos2
        int_pos2 = pos1
        int_q1   = q2
        int_q2   = q1
        int_min1 = min2
        int_min2 = min1
        int_tof1 = min_tof2
        int_tof2 = min_tof1

    return int_pos1, int_pos2, int_q1, int_q2, true_pos1, true_pos2, true_t1, true_t2, int_min1, int_min2, int_tof1, int_tof2

In [21]:
def find_first_time_of_sensors(tof_response, sns_ids):
    #print(tof_response.sensor_id.isin(sns_ids))
    print(sns_ids)
    print(tof_response.sensor_id)
    tof = tof_response[tof_response.sensor_id.isin(sns_ids)]
    #print(tof)
    min_t  = tof.time_bin.min()
    min_df = tof[tof.time_bin == min_t]

    if len(min_df)==0:
        print("Tof dataframe is empty")
        return None, None
    if len(min_df)>1:
        min_id = min_df[min_df.sensor_id == min_df.sensor_id.min()].sensor_id.values[0]
    else:
        min_id = min_df.sensor_id.values[0]

    return np.abs(min_id), min_t

In [22]:
DataSiPM     = db.DataSiPM('petalo', 0)
DataSiPM_idx = DataSiPM.set_index('SensorID')
sns_response = load_mcsns_response(filename)
tof_response = load_mcTOFsns_response(filename)
particles    = load_mcparticles(filename)
hits         = load_mchits(filename)
events       = particles.event_id.unique()

sipms        = DataSiPM_idx.loc[sns_response.sensor_id]
sns_ids      = sipms.index.values

charge_range = (1000, 1400)

for evt in events[31:32]:
    evt_sns = sns_response[sns_response.event_id == evt]
    evt_sns = rf.find_SiPMs_over_threshold(evt_sns, threshold=2)
    if len(evt_sns) == 0:
        continue

    evt_parts = particles   [particles   .event_id == evt]
    evt_hits  = hits        [hits        .event_id == evt]
    evt_tof   = tof_response[tof_response.event_id == evt]

    pos1, pos2, q1, q2, _, _, _, _, min_id1, min_id2, min_tof1, min_tof2 = reconstruct_coincidences(evt_sns, evt_tof, charge_range, DataSiPM_idx, evt_parts, evt_hits)
    
    if len(pos1) and len(pos2):
        print(evt)
        
    #print(pos1, pos2, q1, q2, min_id1, min_id2, min_tof1, min_tof2)

[18446744073709548929 18446744073709548928 18446744073709548926
 18446744073709548925 18446744073709548922 18446744073709548756
 18446744073709548755 18446744073709548754 18446744073709548753
 18446744073709548752 18446744073709548751 18446744073709548750
 18446744073709548749 18446744073709548748 18446744073709548747
 18446744073709548580 18446744073709548578 18446744073709548577
 18446744073709548576 18446744073709548575 18446744073709548574
 18446744073709548573 18446744073709548572 18446744073709548571
 18446744073709548570 18446744073709548406 18446744073709548405
 18446744073709548404 18446744073709548403 18446744073709548402
 18446744073709548401 18446744073709548400 18446744073709548399
 18446744073709548398 18446744073709548397 18446744073709548396
 18446744073709548395 18446744073709548394 18446744073709548232
 18446744073709548231 18446744073709548230 18446744073709548229
 18446744073709548228 18446744073709548227 18446744073709548226
 18446744073709548225 184467440737095482